<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.875 · Deep Learning · PRA
</p>
<p style="margin: 0; text-align:right;">2021-2 · Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# PRA: Recurrent Neural Networks

En esta práctica se implementará diferentes redes neuronales convolucionales para detectar el glaucoma.

**Importante: La entrega debe hacerse en formato notebook y en formato html donde se vea el código y los resultados y comentarios de cada ejercicio. Para exportar el notebook a html puede hacerse desde el menú File $\to$ Download as $\to$ HTML.**

# Autor: Mario Ubierna San Mamés

# Presentación

El objetivo de esta práctica es aplicar los conocimientos adquiridos durante toda la asignatura en un caso clínico real. Para ello se dispondrá de una base de datos que contiene imágenes de ojos sanos y de otros afectados por glaucoma. 

El glaucoma es una patología que afecta al nervio óptico y cuyos orígenes son diversos, es la segunda causa de ceguera por detrás de la diabetes y los efectos en la pérdida de visión son irreversibles. Las causas que lo producen se pueden tratar si la patología es detectada
a tiempo.

El objetivo final de esta práctica es, mediante los conocimientos adquiridos, proponer y entrenar un algoritmo que sea capaz de detectar adecuadamente ojos con glaucoma frente a otros sanos.

# Definición del problema

Los algoritmos de reconocimiento de imágenes se están implementando en la práctica clínica, integrándose en ocasiones directamente en el hardware que se utiliza para la  exploración (por ejemplo, en los ecógrafos). Este tipo de aproximación es lo que se propone en el siguiente artículo científico, el cual se utilizará como base para realizar esta práctica:

*   Diaz-Pinto, A., Morales, S., Naranjo, V. et al. CNNs for automatic glaucoma
assessment using fundus images: an extensive validation. BioMed Eng OnLine
18, 29 (2019). https://doi.org/10.1186/s12938-019-0649-y

En esta práctica se dispone de una serie de imágenes de casos reales. El objetivo es obtener un modelo eficaz para detectar de manera temprana esta patología, reduciendo, por lo tanto, el riesgo de ceguera.

La base de datos está formada por imágenes en color de 224x224 píxeles y se ha
dividido en 10 particiones distintas que se usarán para aplicar un método de cross validation con el objetivo de minimizar errores estadísticos. Cada una de estas particiones, a su vez, contiene tres subconjuntos: train, test y valid. Las imágenes a su vez están etiquetadas de dos formas: normal o abnormal.




# Librerías

In [5]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import _pickle as pickle

import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score

from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras import layers
from keras.models import load_model

import matplotlib.pyplot as plt

# Carga de Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Sección 1 - Análisis exploratorio de los datos

En este apartado se va a realizar diferentes análisis sobre los datos, así se conseguirá entender mejor la problemática que se busca resolver e idearemos un desarrollo para cumplir el objetivo del proyecto.

Pero antes de comenzar con eso hay que realizar la lectura de los datos:

## Lectura de los datos

In [63]:
# Método que carga una porción específica de los datos, según el fold y el tipo de dataset que queremos cargar
def load_dataset(fold="Fold0", dataset="train"):
  root = "/content/drive/MyDrive/practica_DL_UOC_2022/"

  # Cargamos los datos
  df = pd.DataFrame()
  lstFilenames = []
  lstImages = []
  lstTarget = []

  # abnormal (1)
  for filename in os.listdir(os.path.join(root, fold, dataset, "abnormal")):
    lstFilenames.append(filename.split(".")[0]) # Sin .jpg
    lstImages.append(np.array(Image.open(os.path.join("/content/drive/MyDrive/practica_DL_UOC_2022/", fold, dataset, "abnormal", filename))))
    lstTarget.append(1)

  # normal (0)
  for filename in os.listdir(os.path.join(root, fold, dataset, "normal")):
    lstFilenames.append(filename.split(".")[0]) # Sin .jpg
    lstImages.append(np.array(Image.open(os.path.join("/content/drive/MyDrive/practica_DL_UOC_2022/", fold, dataset, "normal", filename))))
    lstTarget.append(0)

  # Devolvemos el resultado
  df = pd.DataFrame(data={
    "FileName": lstFilenames,
    "Image": lstImages,
    "Target": lstTarget
    })
  
  return df

In [64]:
# Método encargado de cargar todos los datos para un Fold
def load_data(fold="Fold0"):

  print("\n" + fold)
  
  # Cargamos el conjunto de train
  print("\tCargando el conjunto de train...")
  df_train = load_dataset(fold=fold, dataset="train")

  # Cargamos el conjunto de validación
  print("\tCargando el conjunto de valid...")
  df_valid = load_dataset(fold=fold, dataset="valid")

  # Cargamos el conjunto de test
  print("\tCargando el conjunto de test...")
  df_test = load_dataset(fold=fold, dataset="test")

  return df_train, df_valid, df_test

El siguiente paso es guardar los dataframes generados para así no tener que hacerlo cada vez:

In [65]:
def save_data_to_pickle(fold="Fold0", df_train=None, df_valid=None, df_test=None):
  root = "/content/drive/MyDrive/practica_DL_UOC_2022/"

  print("\tGuardamos los conjuntos a formato pickle...")
  df_train.to_pickle(os.path.join(root, fold, "df_train.pickle"))
  df_valid.to_pickle(os.path.join(root, fold, "df_valid.pickle"))
  df_test.to_pickle(os.path.join(root, fold, "df_test.pickle"))

Realizamos la lectura dejamos programada la lectura de los ficheros pickle

In [83]:
def load_data_from_pickle(fold="Fold0"):
  root = "/content/drive/MyDrive/practica_DL_UOC_2022/"

  df_train = pd.read_pickle(os.path.join(root, fold, "df_train.pickle"))
  df_valid = pd.read_pickle(os.path.join(root, fold, "df_valid.pickle"))
  df_test = pd.read_pickle(os.path.join(root, fold, "df_test.pickle"))

  return df_train, df_valid, df_test

In [45]:
# df_train, df_valid, df_test = load_data_from_pickle(fold="Fold0")

Lo siguiente es hacer la lectura de todos los folds:

In [67]:
FOLDS = 10

for nFold in range(FOLDS):
  # Obtenemos los datos en un dataframe
  df_train, df_valid, df_test = load_data(fold="Fold" + str(nFold))
  # Guardamos los datos a formato pickle
  save_data_to_pickle(fold="Fold" + str(nFold), df_train=df_train, df_valid=df_valid, df_test=df_test)


Fold0
	Cargando el conjunto de train...
	Cargando el conjunto de valid...
	Cargando el conjunto de test...
	Guardamos los conjuntos a formato pickle...

Fold1
	Cargando el conjunto de train...
	Cargando el conjunto de valid...
	Cargando el conjunto de test...
	Guardamos los conjuntos a formato pickle...

Fold2
	Cargando el conjunto de train...
	Cargando el conjunto de valid...
	Cargando el conjunto de test...
	Guardamos los conjuntos a formato pickle...

Fold3
	Cargando el conjunto de train...
	Cargando el conjunto de valid...
	Cargando el conjunto de test...
	Guardamos los conjuntos a formato pickle...

Fold4
	Cargando el conjunto de train...
	Cargando el conjunto de valid...
	Cargando el conjunto de test...
	Guardamos los conjuntos a formato pickle...

Fold5
	Cargando el conjunto de train...
	Cargando el conjunto de valid...
	Cargando el conjunto de test...
	Guardamos los conjuntos a formato pickle...

Fold6
	Cargando el conjunto de train...
	Cargando el conjunto de valid...
	Cargan

Ahora hay que comprobar que todos los datos estén bien, es decir, que en cada conjunto estén las imágenes correspondientes y que el valor Target sea el que tiene que ser:

In [84]:
# Método que se encarga de comprobar si la lectura está bien
def check_data(fold="Fold0", dataset="train", df=None):
  root = "/content/drive/MyDrive/practica_DL_UOC_2022/"

  lstFilenames = df["FileName"].values

  # abnormal (1)
  for filename in os.listdir(os.path.join(root, fold, dataset, "abnormal")):

    filename = filename.split(".")[0] 
    # Comprobamos que está la imagen
    if not filename in lstFilenames:
      return False
    # Comprobamos que el target es el adecuado
    if df[df["FileName"] == filename]["Target"].item() != 1:
      return False

  # normal (0)
  for filename in os.listdir(os.path.join(root, fold, dataset, "normal")):

    filename = filename.split(".")[0] 
    # Comprobamos que está la imagen
    if not filename in lstFilenames:
      return False
    # Comprobamos que el target es el adecuado
    if df[df["FileName"] == filename]["Target"].item() != 0:
      return False
  
  return True

In [85]:
for nFold in range(FOLDS):
  # Comprobamos la data para cada FOLD
  print("Fold" + str(nFold))
  print("\tLeemos los conjuntos de formato pickle...")
  df_train, df_valid, df_test = load_data_from_pickle(fold="Fold" + str(nFold))

  bTrain = check_data(fold="Fold" + str(nFold), dataset="train", df=df_train)
  print("\tTrain: " + str(bTrain))

  bValid = check_data(fold="Fold" + str(nFold), dataset="valid", df=df_valid)
  print("\tValid: " + str(bValid))

  bTest = check_data(fold="Fold" + str(nFold), dataset="test", df=df_test)
  print("\tTest: " + str(bTest))

Fold0
	Leemos los conjuntos de formato pickle...
	Train: True
	Valid: True
	Test: True
Fold1
	Leemos los conjuntos de formato pickle...
	Train: True
	Valid: True
	Test: True
Fold2
	Leemos los conjuntos de formato pickle...
	Train: True
	Valid: True
	Test: True
Fold3
	Leemos los conjuntos de formato pickle...
	Train: True
	Valid: True
	Test: True
Fold4
	Leemos los conjuntos de formato pickle...
	Train: True
	Valid: True
	Test: True
Fold5
	Leemos los conjuntos de formato pickle...
	Train: True
	Valid: True
	Test: True
Fold6
	Leemos los conjuntos de formato pickle...
	Train: True
	Valid: True
	Test: True
Fold7
	Leemos los conjuntos de formato pickle...
	Train: True
	Valid: True
	Test: True
Fold8
	Leemos los conjuntos de formato pickle...
	Train: True
	Valid: True
	Test: True
Fold9
	Leemos los conjuntos de formato pickle...
	Train: True
	Valid: True
	Test: True


## Análisis

Lo primero de todo es cargar todos los datos para cada uno de los folds, necearios para realizar la práctica:

In [86]:
# Fold0
print("Fold0")
print("\tLeemos los conjuntos de formato pickle...")
f0_df_train, f0_df_valid, f0_df_test = load_data_from_pickle(fold="Fold0")

# Fold1
print("Fold1")
print("\tLeemos los conjuntos de formato pickle...")
f1_df_train, f1_df_valid, f1_df_test = load_data_from_pickle(fold="Fold1")

# Fold2
print("Fold2")
print("\tLeemos los conjuntos de formato pickle...")
f2_df_train, f2_df_valid, f2_df_test = load_data_from_pickle(fold="Fold2")

# Fold3
print("Fold3")
print("\tLeemos los conjuntos de formato pickle...")
f3_df_train, f3_df_valid, f3_df_test = load_data_from_pickle(fold="Fold3")

# Fold4
print("Fold4")
print("\tLeemos los conjuntos de formato pickle...")
f4_df_train, f4_df_valid, f4_df_test = load_data_from_pickle(fold="Fold4")

# Fold5
print("Fold5")
print("\tLeemos los conjuntos de formato pickle...")
f5_df_train, f5_df_valid, f5_df_test = load_data_from_pickle(fold="Fold5")

# Fold6
print("Fold6")
print("\tLeemos los conjuntos de formato pickle...")
f6_df_train, f6_df_valid, f6_df_test = load_data_from_pickle(fold="Fold6")

# Fold7
print("Fold7")
print("\tLeemos los conjuntos de formato pickle...")
f7_df_train, f7_df_valid, f7_df_test = load_data_from_pickle(fold="Fold7")

# Fold8
print("Fold8")
print("\tLeemos los conjuntos de formato pickle...")
f8_df_train, f8_df_valid, f8_df_test = load_data_from_pickle(fold="Fold8")

# Fold9
print("Fold9")
print("\tLeemos los conjuntos de formato pickle...")
f9_df_train, f9_df_valid, f9_df_test = load_data_from_pickle(fold="Fold9")



Fold0
	Leemos los conjuntos de formato pickle...
Fold1
	Leemos los conjuntos de formato pickle...
Fold2
	Leemos los conjuntos de formato pickle...
Fold3
	Leemos los conjuntos de formato pickle...
Fold4
	Leemos los conjuntos de formato pickle...
Fold5
	Leemos los conjuntos de formato pickle...
Fold6
	Leemos los conjuntos de formato pickle...
Fold7
	Leemos los conjuntos de formato pickle...
Fold8
	Leemos los conjuntos de formato pickle...
Fold9
	Leemos los conjuntos de formato pickle...


Una vez cargada toda la información, pasamos a comprobar el número de registros que hay para train, valid y test en cada fold:

In [87]:
for nFold in range(FOLDS):
  df_train, df_valid, df_test = load_data_from_pickle(fold="Fold" + str(nFold))
  print("Fold" + str(nFold) + ":" + "\ttrain(" + str(len(df_train)) + ")" + "\tvalid(" + str(len(df_valid)) + ")" + "\ttest(" + str(len(df_test)) + ")")


Fold0:	train(1379)	valid(154)	test(174)
Fold1:	train(1379)	valid(154)	test(174)
Fold2:	train(1379)	valid(154)	test(174)
Fold3:	train(1379)	valid(154)	test(174)
Fold4:	train(1379)	valid(154)	test(174)
Fold5:	train(1379)	valid(154)	test(174)
Fold6:	train(1379)	valid(154)	test(174)
Fold7:	train(1379)	valid(154)	test(174)
Fold8:	train(1379)	valid(154)	test(174)
Fold9:	train(1379)	valid(154)	test(174)


Como podemos apreciar en la anterior ejecución, todos los folds contienen el mismo número de imágenes tanto para train como para valid y test.

El siguiente punto es comprobar si hay duplicados (imágenes) en cada conjunto, la representación viene de la forma (x) siendo x el número de duplicados en el conjunto:


In [103]:
for nFold in range(FOLDS):
  df_train, df_valid, df_test = load_data_from_pickle(fold="Fold" + str(nFold))
  print("Fold" + str(nFold) + ":" + 
        "\ttrain(" + str(len(df_train[df_train.duplicated(["FileName"])])) + ")" +
        "\tvalid(" + str(len(df_valid[df_valid.duplicated(["FileName"])])) + ")" +
        "\ttest(" + str(len(df_test[df_test.duplicated(["FileName"])])) + ")") 

Fold0:	train(0)	valid(0)	test(0)
Fold1:	train(0)	valid(0)	test(0)
Fold2:	train(0)	valid(0)	test(0)
Fold3:	train(0)	valid(0)	test(0)
Fold4:	train(0)	valid(0)	test(0)
Fold5:	train(0)	valid(0)	test(0)
Fold6:	train(0)	valid(0)	test(0)
Fold7:	train(0)	valid(0)	test(0)
Fold8:	train(0)	valid(0)	test(0)
Fold9:	train(0)	valid(0)	test(0)


Tal y como podemos apreciar, no hay duplicados en ningún conjuto de ningún fold. Por lo que, no se procede a la eliminación de los mismos.

Posteriormente, comprobamos si hay duplicados no dentro de cada conjunto sino que dentro de cada fold, la representación es (x) siendo x el número de duplicados por fold:

In [132]:
for nFold in range(FOLDS):
  df_train, df_valid, df_test = load_data_from_pickle(fold="Fold" + str(nFold))
  lstFilenames = [*df_train["FileName"].values, *df_valid["FileName"].values, *df_test["FileName"].values]
  print("Fold" + str(nFold) + ":" + 
        "\tduplicados(" +  str(len(np.unique(lstFilenames)) - (len(df_train) + len(df_valid) + len(df_test))) + ")"
        ) 

Fold0:	duplicados(0)
Fold1:	duplicados(0)
Fold2:	duplicados(0)
Fold3:	duplicados(0)
Fold4:	duplicados(0)
Fold5:	duplicados(0)
Fold6:	duplicados(0)
Fold7:	duplicados(0)
Fold8:	duplicados(0)
Fold9:	duplicados(0)


Tampoco hay duplicados por fold. Por lo tanto, tampoco se procede a la eliminación de los mismos.

Lo siguiente que vamos a comprobar es el número de casos normal y abnormal que hay en cada conjunto de cada fold, la representación viene de la forma (x,y) siendo x los casos normal e y los abnormal: 

In [133]:
for nFold in range(FOLDS):
  df_train, df_valid, df_test = load_data_from_pickle(fold="Fold" + str(nFold))
  print("Fold" + str(nFold) + ":" + 
        "\ttrain(N:" + str(np.count_nonzero(df_train["Target"].values == 0)) + ", A:" + str(np.count_nonzero(df_train["Target"].values == 1)) +")" +
        "\tvalid(N:" + str(np.count_nonzero(df_valid["Target"].values == 0)) + ", A:" + str(np.count_nonzero(df_valid["Target"].values == 1)) +")" +
        "\ttest(N:" + str(np.count_nonzero(df_test["Target"].values == 0)) + ", A:" + str(np.count_nonzero(df_test["Target"].values == 1)) +")")

Fold0:	train(N:754, A:625)	valid(N:83, A:71)	test(N:82, A:92)
Fold1:	train(N:740, A:639)	valid(N:88, A:66)	test(N:91, A:83)
Fold2:	train(N:739, A:640)	valid(N:83, A:71)	test(N:97, A:77)
Fold3:	train(N:743, A:636)	valid(N:85, A:69)	test(N:91, A:83)
Fold4:	train(N:746, A:633)	valid(N:81, A:73)	test(N:92, A:82)
Fold5:	train(N:758, A:621)	valid(N:71, A:83)	test(N:90, A:84)
Fold6:	train(N:754, A:625)	valid(N:84, A:70)	test(N:81, A:93)
Fold7:	train(N:737, A:642)	valid(N:82, A:72)	test(N:100, A:74)
Fold8:	train(N:748, A:631)	valid(N:80, A:74)	test(N:91, A:83)
Fold9:	train(N:733, A:646)	valid(N:82, A:72)	test(N:104, A:70)


Como podemos apreciar no hay el mismo número de casos en ningún conjunto para cada fold.

Por lo que vamos a comprobar el número de casos normales y abnormal por fold, independientemente del conjunto que sea, la representación sigue siendo igual (x,y), x son los casos normal e y los abnormal:

In [134]:
for nFold in range(FOLDS):
  df_train, df_valid, df_test = load_data_from_pickle(fold="Fold" + str(nFold))
  nNormal = np.count_nonzero(df_train["Target"].values == 0) + np.count_nonzero(df_valid["Target"].values == 0) + np.count_nonzero(df_test["Target"].values == 0)
  nAbnormal = np.count_nonzero(df_train["Target"].values == 1) + np.count_nonzero(df_valid["Target"].values == 1) + np.count_nonzero(df_test["Target"].values == 1)
  print("Fold" + str(nFold) + ":" + 
        "\tcases(N:" + str(nNormal) + ", A:" + str(nAbnormal) + ")"
        ) 

Fold0:	cases(N:919, A:788)
Fold1:	cases(N:919, A:788)
Fold2:	cases(N:919, A:788)
Fold3:	cases(N:919, A:788)
Fold4:	cases(N:919, A:788)
Fold5:	cases(N:919, A:788)
Fold6:	cases(N:919, A:788)
Fold7:	cases(N:919, A:788)
Fold8:	cases(N:919, A:788)
Fold9:	cases(N:919, A:788)


Tal y como podemos observar todos los folds tienen el mismo número de casos.

Con esto finalizamos este punto, destacar que en el informe se incluirán gráficas hechas con [Infogram](https://infogram.com/). Básicamente, porque una vez tenemos los datos que nos interesa es más fácil crear las gráficas y que tengan un aspecto elegante. Estas gráficas se pueden visualizar en el informe entregado con la práctica.

# Sección 2 - Entrenamiento de la red sobre Fold0